In [170]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np
# from geographiclib.geodesic import Geodesic
from shapely.geometry import *
# import simplekml
import geopandas as gpd
from app import main as m
import app.main
import app.netools as ne
nt = ne.NetTools()
import app.mydef as md
mt = md.MyTools()

# 判断小区是否可以优化解决mr问题

In [8]:
ditu_cell = pd.read_csv(open('/mypy/课题/优化/武汉基站数据库.csv',encoding = 'gbk'))

In [173]:
ditu_cell=ditu_cell.loc[(ditu_cell['RRU经度']<130)&(ditu_cell['RRU纬度']<40)&(ditu_cell['RRU纬度']>20)]

In [174]:
ditu_cell['小区中文名'] = ditu_cell['小区中文名'].fillna('无')

In [235]:
ditu_cell=ditu_cell[ ~ditu_cell['小区中文名'].str.contains('r')]  #删除某列包含特殊字符，过滤删除，筛选剔除

In [176]:
ditu_cell=ditu_cell[ ~ ditu_cell['小区中文名'].str.contains('m-')]  #删除某列包含特殊字符的行

In [177]:
ditu_cell=ditu_cell[ ~ ditu_cell['小区中文名'].str.contains('w-')]  #删除某列包含特殊字符的行

In [178]:
ditu_cell=ditu_cell.dropna(subset=['RRU经度','RRU纬度'])#

In [179]:
ditu_cell.shape

(2796, 9)

In [180]:
wt_cell = pd.read_csv(open('/mypy/课题/优化/武汉问题小区.csv',encoding = 'gbk'))

In [181]:
wt_cell['小区中文名'] = wt_cell['小区中文名'].fillna('无')

In [182]:
wt_cell.columns

Index(['小区CGI(*)', '小区中文名', '网络制式(TDD/FDD)', 'TDL规划站号(新)', '所属地区', '工程参数标识',
       '区县代码', '基站编码', 'VIP基站', '入网工程期', 'ATU网格', 'RRU经度', 'RRU纬度', '天线挂高',
       '地面海拔', '入网时间', '2G共址站名', '与FDD1800共址的FDD900站名', '站址归属单位', '覆盖类型',
       '覆盖类别', '覆盖场景', '场景名称', 'VOLTE场景', '方位角', '机械下倾角', '电子下倾角', '是否电调天线',
       '电下倾天线类型', '天线厂家', '工作频段', '设备型号', '设备类型(大类)', '分布式或一体化', '小区所属空间资源',
       '基站所属空间资源', 'eNodeB软件版本', '所属OMC', '归属区县', '厂家名称', '物理站名', '集中开站所在环节',
       '集中开站工单号', '姿态仪测量', '姿态仪测量时间', '所属eNodeB名称', '采集经度', '采集纬度', 'RRU数量',
       'TAC', 'PCI', '中心载频的信道号', '频段指示', '空口上下行业务字节数KByte', '统计分类'],
      dtype='object')

In [236]:
wt_cell_x = wt_cell[['小区CGI(*)','小区中文名']]

In [184]:
res=ditu_cell.merge(wt_cell_x,how='left', on='小区CGI(*)',suffixes=('', '_y'),indicator=True)

In [185]:
ditu_cell = res.loc[res['_merge']=='both']

In [186]:
ditu_cell.columns

Index(['小区CGI(*)', '小区中文名', '网络制式(TDD/FDD)', 'RRU经度', 'RRU纬度', '天线挂高', '方位角',
       '归属区县', '是否故障', '小区中文名_y', '_merge'],
      dtype='object')

In [187]:
def juli(df,changjing,pinduan):
    
    F900 = {0:'300',1:'430',2:'430',3:'430',4:'1200',5:'2200'}
    TDD = {0:'250',1:'400',2:'400',3:'350',4:'800',5:'1000'}
    
    df['juli']= [F900[cj] if pd == 'FDD900' else TDD[cj] 
                 for cj , pd in zip(df[changjing],df[pinduan])]
    return df

In [188]:
wt_cell["覆盖类别"]=wt_cell["覆盖类别"].fillna(3)#na的用3填充

In [189]:
wt_cell1 = juli(wt_cell,'覆盖类别','工作频段')

In [190]:
wt_cell1.shape

(1664, 56)

In [191]:
wt_cell1=wt_cell.dropna(subset=['RRU经度','RRU纬度','方位角','juli'])#

In [192]:
wt_cell1.shape

(1661, 56)

In [193]:
wt_cell1=wt_cell1.loc[wt_cell1['RRU经度']<117]

In [194]:
wt_cell1.shape

(1661, 56)

In [195]:
wt_cell12 = wt_cell1.copy()

In [196]:
wt_cell12['RRU经度'] = pd.to_numeric(wt_cell12['RRU经度'])

wt_cell12['RRU纬度'] = pd.to_numeric(wt_cell12['RRU纬度'])

wt_cell12['方位角'] = pd.to_numeric(wt_cell12['方位角'])

wt_cell12['juli'] = pd.to_numeric(wt_cell12['juli'])

In [197]:
wt_cell12[['RRU经度','RRU纬度','方位角','juli']].isnull().sum().sum()

0

## 周边存在现网站点

In [198]:
shanqu=nt.add_sectors_jl(wt_cell12, coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'juli'], 
                         has_z=False,shape_dict={'beam': 120, 'per_degree': 10})

In [199]:
dian_ditu = nt.add_points(ditu_cell,coords=['RRU经度', 'RRU纬度','',''])

In [200]:
wt_cell12.shape

(1661, 56)

In [201]:
res_zong = gpd.sjoin(shanqu,dian_ditu)

In [202]:
res_zong['优先级']= 0
res_zong['方案类型']= '无'

In [203]:
res_zong.shape

(11884, 71)

In [204]:
res_zong_jl=mt.distancea_df(res_zong,'RRU经度_left','RRU纬度_left','RRU经度_right', 'RRU纬度_right')

In [205]:
res_zong_jl = res_zong_jl.reset_index(drop = True)

In [206]:
res_zong_jl.shape

(11884, 72)

In [207]:
#删除距离小于25米，现网站点网络制式不是FDD的小区。
res_zong_j2 = res_zong_jl.drop(res_zong_jl[(res_zong_jl['距离'] < 25) 
                                           & (res_zong_jl['网络制式(TDD/FDD)_right'] !='FDD')].index) #多条件删除，符合条件的行

In [208]:
res_zong_j2.shape

(9198, 72)

In [209]:
#删除距离小于25米，现网站点和问题小区网络制式都为FDD的小区
res_zong_j3 = res_zong_j2.drop(res_zong_j2[(res_zong_j2['距离'] < 25) 
                                           & (res_zong_j2['网络制式(TDD/FDD)_right'] =='FDD')  
                                           & (res_zong_j2['网络制式(TDD/FDD)_left'] =='FDD')].index) #多条件删除，符合条件的行

In [210]:
res_zong_j3.shape

(7124, 72)

In [211]:
res_zong_j3.loc[res_zong_j3['网络制式(TDD/FDD)_right'].str.contains('FDD'),'优先级']= 2 #过滤条件并更新值

In [212]:
res_zong_j3.loc[res_zong_j3['网络制式(TDD/FDD)_right'].str.contains('TDD'),'优先级']= 3 #过滤条件并更新值

In [213]:
res_zong_j3.loc[res_zong_j3['天线挂高_right']<30,'优先级']= 4 #过滤条件并更新值

In [214]:
res_zong_j3.loc[res_zong_j3['天线挂高_right']<25,'优先级']= 5 #过滤条件并更新值

In [215]:
res_zong_j3.loc[res_zong_j3['天线挂高_left']<25,'优先级']= 6 #过滤条件并更新值

In [216]:
res_zong_j3.loc[res_zong_j3['小区中文名_right'].str.contains('美化'),'优先级']= 7 #过滤条件并更新值

In [217]:
res_zong_j3.loc[res_zong_j3['小区中文名_left'].str.contains('美化'),'优先级']= 8 #过滤条件并更新值

In [219]:
res_zong_j3['优先级'] = res_zong_j3['优先级'].mask(res_zong_j3['优先级']<1,1)

In [220]:
res_zong_j3 = res_zong_j3.sort_values(by="优先级")#按照优先级排序默认升序

In [221]:
res_zong_j3.shape

(7124, 72)

In [222]:
res_zong_youhua = res_zong_j3.drop_duplicates(subset='小区CGI(*)_left')#删除重复项

In [223]:
res_zong_youhua.shape

(815, 72)

In [224]:
res_zong_youhua.to_csv('res.csv',encoding = 'gbk',index = False)

## 有故障站

In [229]:
res_zong_jl['是否故障'] = res_zong_jl['是否故障'].fillna('否')

In [230]:
res_zong_jl.loc[(res_zong_jl['是否故障'].str.contains('是')),'方案类型'] = '周边小区故障'

In [231]:
res_zong_jl.loc[(res_zong_jl['统计分类'].str.contains('故障')),'方案类型'] = '问题小区故障'

In [232]:
res_zong_guzhang=res_zong_jl.loc[(res_zong_jl['方案类型']== '问题小区故障') | (res_zong_jl['方案类型']== '周边小区故障')]

In [233]:
res_zong_guzhang.shape

(171, 72)

In [234]:
res_zong_guzhang.to_csv('res.csv',encoding = 'gbk',index = False)

## 过高站点下倾角小

## 过矮站点下倾角大 

# ^^^结束